# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.Connection('data.sqlite')

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [4]:
q= """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
"""
pd.read_sql(q, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [3]:
q = """
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN (SELECT customerNumber
                     FROM orders
                     WHERE orderDate = "2003-01-31")
;
"""
pd.read_sql(q, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [9]:
q= """
SELECT COUNT(orderNumber) as num_orders, productName
FROM orderdetails
JOIN products
USING(productCode)
GROUP BY productCode
ORDER BY num_orders DESC
;
"""
pd.read_sql(q, conn)

,num_orders,productName
0,53,1992 Ferrari 360 Spider red
1,28,Boeing X-32A JSF
2,28,American Airlines: MD-11S
3,28,F/A 18 Hornet 1/72
4,28,ATA: B757-300
...,...,...
104,25,1911 Ford Town Car
105,25,1917 Grand Touring Sedan
106,25,1965 Aston Martin DB5
107,24,1952 Citroen-15CV


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [11]:
q = """
SELECT COUNT(DISTINCT customerNumber) as num_customers, productName
FROM orderdetails
JOIN products
USING(productCode)
JOIN orders
USING(orderNumber)
GROUP BY productCode
ORDER BY num_customers DESC
;
"""
pd.read_sql(q, conn)

,num_customers,productName
0,40,1992 Ferrari 360 Spider red
1,27,Boeing X-32A JSF
2,27,1934 Ford V8 Coupe
3,27,1972 Alfa Romeo GTA
4,27,1952 Alpine Renault 1300
...,...,...
104,19,1958 Chevy Corvette Limited Edition
105,18,2002 Chevy Corvette
106,18,1969 Chevrolet Camaro Z28
107,18,1952 Citroen-15CV


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [17]:
q = """
SELECT COUNT(DISTINCT customerNumber) as num_customers, productName, employeeNumber, firstName, lastName, offices.city, offices.officeCode
FROM orderdetails
JOIN products
USING(productCode)
JOIN orders
USING(orderNumber)
JOIN customers
USING(customerNumber)
JOIN employees
ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN offices
USING(officeCode)
GROUP BY productCode
HAVING num_customers < 20
;
"""
pd.read_sql(q, conn)

,num_customers,productName,employeeNumber,firstName,lastName,city,officeCode
0,18,1949 Jaguar XK 120,1501,Larry,Bott,London,7
1,19,1958 Chevy Corvette Limited Edition,1370,Gerard,Hernandez,Paris,4
2,18,1952 Citroen-15CV,1501,Larry,Bott,London,7
3,18,1969 Chevrolet Camaro Z28,1501,Larry,Bott,London,7
4,18,2002 Chevy Corvette,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [19]:
q = """
SELECT AVG(creditLimit) as avg_credit, employeeNumber, firstName, lastName, COUNT(customerNumber) as num_customers
FROM customers
JOIN employees
ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY employeeNumber
HAVING avg_credit > 15000
;
"""
pd.read_sql(q, conn)

,avg_credit,employeeNumber,firstName,lastName,num_customers
0,100433.333333,1165,Leslie,Jennings,6
1,65266.666667,1166,Leslie,Thompson,6
2,73916.666667,1188,Julie,Firrelli,6
3,81533.333333,1216,Steve,Patterson,6
4,66614.285714,1286,Foon Yue,Tseng,7
5,80887.500000,1323,George,Vanauf,8
6,86233.333333,1337,Loui,Bondur,6
7,91785.714286,1370,Gerard,Hernandez,7
8,81340.000000,1401,Pamela,Castillo,10
9,91187.500000,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!